In [2]:
# Let's assume we are building a food delivery app, and we need to manage the different states of an order.
# The order can transition between multiple states, such as placed, preparing, out for delivery, and delivered.

class Order:
    def __init__(self):
        # Constructor initializes the state to ORDER_PLACED
        self.state = "ORDER_PLACED"

    # Method to cancel the order
    # only allows cancellation if in ORDER_PLACED or PREPARING states
    def cancelOrder(self):
        if self.state == "ORDER_PLACED" or self.state == "PREPARING":
            self.state = "CANCELLED"
            print("Order has been cancelled.")
        else:
            print("Cannot cancel the order now.")

    # Method to move the order to the next state based on its current state
    def nextState(self):
        if self.state == "ORDER_PLACED":
            self.state = "PREPARING"
        elif self.state == "PREPARING":
            self.state = "OUT_FOR_DELIVERY"
        elif self.state == "OUT_FOR_DELIVERY":
            self.state = "DELIVERED"
        else:
            print(f"No next state from: {self.state}")
            return
        print(f"Order moved to: {self.state}")

    # Getter for the state
    def getState(self):
        return self.state


# Main function to test the order flow
if __name__ == "__main__":
    order = Order()

    # Display initial state
    print("Initial State:", order.getState())

    # Moving through states
    order.nextState()  # ORDER_PLACED -> PREPARING
    order.nextState()  # PREPARING -> OUT_FOR_DELIVERY
    order.nextState()  # OUT_FOR_DELIVERY -> DELIVERED

    # Attempting to cancel an order after it is out for delivery
    order.cancelOrder()  # Should not allow cancellation

    # Display final state
    print("Final State:", order.getState())


# State Transition Management:
# The state transitions are hardcoded in the nextState() method using a switch statement. This approach becomes cumbersome if new states need to be added.

# Lack of Encapsulation:
# The state transition logic and cancel behavior are directly handled within the Order class. This violates the Single Responsibility Principle by combining multiple responsibilities within a single class.

# Code Duplication:
# The logic for the cancelOrder() and nextState() methods could lead to duplicate logic if more states and actions are added.

# Missing Flexibility for Future Changes:
# Adding new states or changing existing behaviors can be error-prone and cumbersome, as the Order class needs to be updated each time.


Initial State: ORDER_PLACED
Order moved to: PREPARING
Order moved to: OUT_FOR_DELIVERY
Order moved to: DELIVERED
Cannot cancel the order now.
Final State: DELIVERED


In [3]:
# OrderContext class manages the current state of the order
class OrderContext:
    def __init__(self):
        self.currentState = OrderPlacedState()  # default state

    # Method to set a new state for the order
    def setState(self, state):
        self.currentState = state

    # Method to move the order to the next state
    def next(self):
        self.currentState.next(self)

    # Method to cancel the order
    def cancel(self):
        self.currentState.cancel(self)

    # Method to get the current state of the order
    def getCurrentState(self):
        return self.currentState.getStateName()


# OrderState interface defines the behavior of the order states
class OrderState:
    def next(self, context):  # Move to the next state
        pass

    def cancel(self, context):  # Cancel the order
        pass

    def getStateName(self):  # Get the name of the state
        pass


# Concrete states for each stage of the order

# OrderPlacedState handles the behavior when the order is placed
class OrderPlacedState(OrderState):
    def next(self, context):
        context.setState(PreparingState())
        print("Order is now being prepared.")

    def cancel(self, context):
        context.setState(CancelledState())
        print("Order has been cancelled.")

    def getStateName(self):
        return "ORDER_PLACED"


# PreparingState handles the behavior when the order is being prepared
class PreparingState(OrderState):
    def next(self, context):
        context.setState(OutForDeliveryState())
        print("Order is out for delivery.")

    def cancel(self, context):
        context.setState(CancelledState())
        print("Order has been cancelled.")

    def getStateName(self):
        return "PREPARING"


# OutForDeliveryState handles the behavior when the order is out for delivery
class OutForDeliveryState(OrderState):
    def next(self, context):
        context.setState(DeliveredState())
        print("Order has been delivered.")

    def cancel(self, context):
        print("Cannot cancel. Order is out for delivery.")

    def getStateName(self):
        return "OUT_FOR_DELIVERY"


# DeliveredState handles the behavior when the order is delivered
class DeliveredState(OrderState):
    def next(self, context):
        print("Order is already delivered.")

    def cancel(self, context):
        print("Cannot cancel a delivered order.")

    def getStateName(self):
        return "DELIVERED"


# CancelledState handles the behavior when the order is cancelled
class CancelledState(OrderState):
    def next(self, context):
        print("Cancelled order cannot move to next state.")

    def cancel(self, context):
        print("Order is already cancelled.")

    def getStateName(self):
        return "CANCELLED"


# Main method to test the order flow
if __name__ == "__main__":
    order = OrderContext()

    # Display initial state
    print("Current State:", order.getCurrentState())

    # Moving through states
    order.next()  # ORDER_PLACED -> PREPARING
    order.next()  # PREPARING -> OUT_FOR_DELIVERY
    order.cancel()  # Should fail, as order is out for delivery
    order.next()  # OUT_FOR_DELIVERY -> DELIVERED
    order.cancel()  # Should fail, as order is delivered

    # Display final state
    print("Final State:", order.getCurrentState())


Current State: ORDER_PLACED
Order is now being prepared.
Order is out for delivery.
Cannot cancel. Order is out for delivery.
Order has been delivered.
Cannot cancel a delivered order.
Final State: DELIVERED
